In [1]:
from fastai.vision.all import *
from fastai.vision.widgets import *
from ipywidgets import *
import jaconv
import pathlib
plt = platform.system()
if plt == 'Linux': pathlib.WindowsPath = pathlib.PosixPath
out_pl = widgets.Output()
btn_upload = widgets.FileUpload()
replace_dic = {"_ッロウッ":" (group)","シボリニア_ッラシリペッ":"キボリニア・グラキリペス","ダッリョ":"ダクリオ","ボタニボタケ":"ボタンイボタケ","オホニテングタケ":"オオオニテングタケ","オファライタケ":"オオワライタケ","オリヴェ":"オリーブ"}

In [2]:
path = Path()
import gdown
url = "https://drive.google.com/u/1/uc?id=1Q7jugVCIZfojGYs_ikyJG6rqXXPhRzYF&export=download"
file_name = "kinoko-uranai.pkl"
gdown.download(url, file_name)
model_inf = load_learner(path / file_name, cpu=True)
# model_inf = load_learner(path/"all_20210829_res34.pkl", cpu=True)

Downloading...
From: https://drive.google.com/u/1/uc?id=1Q7jugVCIZfojGYs_ikyJG6rqXXPhRzYF&export=download
To: C:\Users\Atsushi Nakajima\Dropbox\notebooks\kinoko-uranai.pkl
90.1MB [00:07, 11.3MB/s]


In [3]:
def on_data_change(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(448,448))
    result_dic = {}
    pred_class, pred_idxs, outputs = model_inf.predict(img)
    top_5_conf, i = outputs.topk(5)
    itr = 0
    classes = model_inf.dls.vocab
    buf = []
    for x in i:
        kwamei = jaconv.alphabet2kata(classes[x.item()].lower())
        for k,v in replace_dic.items():
            kwamei = kwamei.replace(k,v)
        proba = round(top_5_conf[itr].item(),3)
        if proba != 0:
            buf.append(kwamei+" "+str(proba*100)[:4]+"%")
        itr=itr+1
    buf = " / ".join(buf)
    htmlWidget = widgets.HTML(value = f"<span style='color:steelblue;font-size:medium;'><b>【今日のラッキーきのこ】</b></span>")        
    with out_pl: display(htmlWidget)
    htmlWidget = widgets.HTML(value = f"<span style='color:maroon;font-size:large;'>{buf}</span>")
    with out_pl: display(htmlWidget) 

In [4]:
btn_upload.observe(on_data_change, names=['data'])
display(VBox([widgets.Label("きのこうらない"), btn_upload, out_pl]))